Do the regression with another outcome variable: work effort, where we take workin overtime as the measure ("Do you work overtime?" -> Yes/No)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import math

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

In [2]:
# define path: insert the path where the SOEP data is stored on your computer here
from pathlib import Path
data_folder = Path("/Volumes/dohmen_soep/SOEP-CORE.v36eu_STATA/Stata/raw")
# define relevant subsets of SOEP-data
file_names = ['vp', 'wp']

file_paths = [data_folder / f"{file_name}.dta" for file_name in file_names]
# some controls are in gen data
file_paths_2 = [data_folder / f"{file_name}gen.dta" for file_name in file_names]

In [3]:
data05 = pd.read_stata(file_paths[0], columns=["pid","hid", "syear","vp12602", "vp12603", "vp12605"]).set_index(['pid', 'hid'])
df_05 = data05.rename(columns={ 'vp12602': 'take_revenge', 'vp12603': 'similar_problems', 'vp12605': 'insult_back'})

In [21]:

data06 = pd.read_stata(file_paths[1], columns=["pid", "hid", "syear","wp12402", "wp12401", "wp55", "wp5902", 'wp43b01', "wp43b03", "wp43b05", "wp43b07"]).set_index(['pid', 'hid'])
df_06 = data06.rename(columns={"wp12401": "gender", "wp12402": "year_birth", 'wp43b01': "recog_sup", "wp43b03": "recog_effort", "wp43b05": "recog_personal", "wp43b07": "recog_pay", "wp5902": "wage_lastmonth", "wp55": "overtime"})
#'wp43b02': 'felt_recog_sup', 'wp43b04': 'felt_recog_effort', 'wp43b06': 'felt_recog_pay', "wp43b03": "recog_effort", "wp43b05": "recog_personal", "wp43b07": "recog_pay"
#"wp0102": "satisfaction_work"

In [59]:

hours06 = pd.read_stata(file_paths_2[1], columns=["pid","hid", "syear", "wvebzeit", "betr06", "wpsbil", "nace06", "werwzeit", "wbilzeit"]).set_index(['pid', 'hid'])
work06 = hours06.rename(columns={'wvebzeit': 'working_hours', "betr06": "firmsize", "wpsbil": "school_degree", "nace06": "sector", "werwzeit": "tenure" , "wbilzeit" : "years_educ"})

In [61]:
# mapping for reciprocity questions: same scale for all
reciprocity_questions_mapping = {
    '[1] Trifft ueberhaupt nicht zu': 1,
    '[2] Skala 1-7': 2,
    '[3] Skala 1-7': 3,
    '[4] Skala 1-7': 4,
    '[5] Skala 1-7': 5,
    '[6] Skala 1-7': 6,
    '[7] Trifft voll zu': 7,
    '[-1] keine Angabe': -1,
}

overtime_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Ja': 1,
    '[2] Nein': 2,
    '[3] T.n.z., Selbstaendiger': 3,
}
## mapping for recognition questions: binary
recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Ja': 1,
    '[2] Nein': 2,
}
# felt recog mapping
felt_recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Gar nicht': 1,
    '[2] Maessig': 2,
    '[3] Stark': 3,
    '[4] Sehr stark': 4,
}
# mapping for firmsize -> we need to recode this in a sensible way: jumps are the same: first change: selbstständig to 0
firmsize_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Unter  5': 1,
    '[2] 5 bis 10': 2,
    '[3] 11 bis unter 20': 3,
    '[4] bis 90: unter 20': 4,
    '[5] 91-04: 5 bis unter 20': 5,
    '[6] 20 bis unter 100': 6,
    '[7] 100 bis unter 200': 7,
    '[8] bis 98: 20 bis unter 200': 8,
    '[9] 200 bis unter 2000': 9,
    '[10] 2000 und mehr': 10,
    '[11] Selbstaendig-ohne Mitarb.': 0,
}

# mapping for sectors: only to easier remove negatives
sector_map = {
    "[1] Landwirtschaft und  Jagd": 1,
    "[2] Forstwirtschaft": 2,
    "[5] Fischerei und Fischzucht": 5,
    "[10] Kohlenbergbau, Torfgewinnung": 10,
    "[11] Gewinnung von Erdöl und Erdgas, Erbringung damit verbundener Dienstleistungen": 11,
    "[12] Bergbau auf Uran- und Thoriumerze": 12,
    "[13] Erzbergbau": 13,
    "[14] Gewinnung von Steinen und Erden, sonstiger Bergbau": 14,
    "[15] Herstellung von Nahrungs- und Futtermitteln sowie Getränken": 15,
    "[16] Tabakverarbeitung": 16,
    "[17] Herstellung von Textilien": 17,
    "[18] Herstellung von Bekleidung": 18,
    "[19] Herstellung von Leder und Lederwaren": 19,
    "[20] Herstellung von Holz sowie Holz-, Kork- und Flechtwaren (ohne Herstellung von Möbeln)": 20,
    "[21] Herstellung von Papier, Pappe und Waren daraus": 21,
    '[22] Herstellung von Verlags- und Druckerzeugnissen,  Vervielfältigung von bespielten Ton-, Bild- und Datenträgern': 22,
    "[23] Kokerei, Mineralölverarbeitung, Herstellung und Verarbeitung von Spalt- und Brutstoffen": 23,
    "[24] Herstellung von chemischen Erzeugnissen": 24,
    "[25] Herstellung von Gummi- und Kunststoffwaren": 25,
    "[26] Herstellung von Glas und Glaswaren, Keramik, Verarbeitung von Steinen und Erden": 26,
    "[27] Metallerzeugung und -bearbeitung": 27,
    "[28] Herstellung von Metallerzeugnissen": 28,
    "[29] Maschinenbau": 29,
    "[31] Herstellung von Geräten der Elektrizitätserzeugung, -verteilung u. Ä.": 31,
    "[30] Herstellung von Büromaschinen, Datenverarbeitungsgeräten und -einrichtungen": 30,
    "[32] Rundfunk- und Nachrichtentechnik": 32,
    "[33] Medizin-, Mess-, Steuer- und Regelungstechnik, Optik, Herstellung von Uhren": 33,
    "[34] Herstellung von Kraftwagen und Kraftwagenteilen": 34,
    "[35] Sonstiger Fahrzeugbau": 35,
    "[36] Herstellung von Möbeln, Schmuck, Musikinstrumenten, Sportgeräten, Spielwaren und sonstigen Erzeugnissen": 36,
    "[37] Rückgewinnung": 37,
    "[40] Energieversorgung": 40,
    "[41] Wasserversorgung": 41,
    "[45] Bau": 45,
    "[50] Kraftfahrzeughandel; Instandhaltung und Reparatur von Kraftfahrzeugen; Tankstellen": 50,
    "[51] Handelsvermittlung und Großhandel (ohne Handel mit Kraftfahrzeugen)": 51,
    "[52] Einzelhandel (ohne Handel mit Kraftfahrzeugen und ohne Tankstellen); Reparatur von Gebrauchsgütern": 52,
    "[55] Beherbergungs- und Gaststätten": 55,
    "[60] Landverkehr; Transport in Rohrfernleitungen": 60,
    "[61] Schifffahrt": 61,
    "[62] Luftfahrt": 62,
    "[63] Hilfs- und Nebentätigkeiten für den Verkehr; Verkehrsvermittlung": 63,
    "[64] Nachrichtenübermittlung": 64,
    "[65] Kreditinstitute": 65,
    "[66] Versicherungen (ohne Sozialversicherung)": 66,
    "[67] Mit den Kreditinstituten und Versicherungen verbundene Tätigkeiten": 67,
    "[70] Grundstücks- und Wohnungswesen": 70,
    "[71] Vermietung beweglicher Sachen ohne Bedienungspersonal": 71,
    "[72] Datenverarbeitung und Datenbanken": 72,
    "[73] Forschung und Entwicklung": 73,
    "[74] Erbringung von unternehmensbezogenen Dienstleistungen": 74,
    "[75] Öffentliche Verwaltung, Verteidigung, Sozialversicherung": 75,
    "[80] Erziehung und Unterricht": 80,
    "[85] Gesundheits-, Veterinär- und Sozialwesen": 85,
    "[90] Abwasser- und Abfallbeseitigung und sonstige Entsorgung": 90,
    "[91] Interessenvertretungen sowie kirchliche und sonstige Vereinigungen (ohne Sozialwesen, Kultur und Sport)": 91,
    "[92] Kultur, Sport und Unterhaltung": 92,
    "[93] Erbringung von sonstigen Dienstleistungen": 93,
    "[95] Private Haushalte mit Hauspersonal": 95,					
    "[96] Industrie - ohne weitere Zuordnung": 96,					
    "[97] Handwerk - ohne weitere Zuordnung": 97,					
    "[98] Dienstleistungen ohne weitere Zuordnung": 98,					
    "[99] Exterritoriale Organisationen und Körperschaften": 99,				
    "[100] Produzierendes Gewerbe ohne w.Zuordnung": 100,
    "[-1] keine Angabe": -1,
    '[-2] trifft nicht zu': -2, 
    "[-3] unplausibler Wert": -3,
    "[-4] unzulaessige Mehrfachantwort": -4, 
    "[-5] in Fragebogenversion nicht enthalten": -5,
    "[-6] Fragebogenversion mit geaenderter Filterfuehrung": -6, 
    "[-7] nur in weniger eingeschraenkter Edition verfuegbar": -7,
    "[-8] Frage in diesem Jahr nicht Teil des Frageprogramms": -8,
}
# mapping for school degree: to easier remove negatives
school_degree_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[1] Hauptschulabschluss': 1,
    '[2] Realschulabschluss': 2,
    '[3] Fachhochschulreife': 3,
    '[4] Abitur': 4,
    '[5] Anderer Abschluss': 5,
    '[6] Ohne Abschluss verlassen': 6,
    '[7] Noch kein Abschluss': 7,
    '[8] Keine Schule besucht': 8,
}
# reversed mapping to redo changes

reversed_mapping_sector = {v: k for k, v in sector_map.items()}
reversed_mapping_schoold = {v: k for k, v in school_degree_mapping.items()}

In [62]:

def recode_categoricals(inputdf):

    merged = inputdf  
    
    # recode Gender variable
    merged['gender'].replace('[2] Weiblich', 2,inplace=True)
    merged['gender'].replace('[1] Maennlich', 1,inplace=True)
    # recode reciprocity variables
    merged[["similar_problems","take_revenge","insult_back"]] = merged[["similar_problems","take_revenge","insult_back"]].apply(lambda x: x.map(reciprocity_questions_mapping))
    # recode recognition variables
    merged[["recog_sup","recog_effort","recog_personal","recog_pay"]] = merged[["recog_sup","recog_effort","recog_personal","recog_pay"]].apply(lambda x: x.map(recog_mapping))
    # recode felt recognition variables
    #merged[["felt_recog_sup","felt_recog_effort","felt_recog_personal","felt_recog_pay"]] = merged[["felt_recog_sup","felt_recog_effort","felt_recog_personal","felt_recog_pay"]].apply(lambda x: x.map(felt_recog_mapping))
    # recode firm size
    merged['firmsize'] = merged['firmsize'].map(firmsize_mapping)
    # recode new job reason variable
   # merged['reason_new_job'] = merged['reason_new_job'].map(reason_new_job_mapping)
    # recode job change variable
   # merged['new_job']= merged['new_job'].map(new_job_mapping)
    # recode turnover intention variable
    #merged['turnover_intention'] = merged['turnover_intention'].map(turnover_mapping)
    # recode sector
    merged['sector'] = merged['sector'].map(sector_map)
    # recode school degree
    merged['school_degree'] = merged['school_degree'].map(school_degree_mapping)
    merged['overtime'] = merged['overtime'].map(overtime_mapping)
    output = merged
    return output

In [63]:
# Merge dataframes: a bit tough to read as its nested, merges 3 dataframes: 2005,2006,and 2006gen
allmerged_df = pd.merge(pd.merge(df_05, df_06, on=['pid', 'hid']), work06, on=['pid', 'hid'])
recoded = recode_categoricals(allmerged_df).astype('int')

# construct indsutry-relative wage
sector_wage_averages = recoded.groupby('sector')['wage_lastmonth'].mean()
recoded["sector_avg_wage"] = recoded["sector"].map(sector_wage_averages)
recoded["relative_wage"] = recoded["wage_lastmonth"] / recoded["sector_avg_wage"]
# replaces negative values with n.a.n 
recoded = recoded.mask(recoded < 0, np.nan) 
# construct avg reciprocity measure
recoded['avg_rec'] = recoded[['take_revenge', 'similar_problems', 'insult_back']].mean(axis=1)
# construct age, potential experience and age^2
recoded['age'] = 2006 - recoded['year_birth']
recoded["potential_experience"] = pow((recoded["age"] - 18), 2)
recoded["age_squared"] = (recoded["age"] ** 2) / 100
# recode categoricals back to make it better readable
#recoded["reason_new_job"] = recoded["reason_new_job"].map(reversed_mapping_reason)
recoded["sector"]=recoded["sector"].map(reversed_mapping_sector)
recoded["school_degree"] = recoded["school_degree"].map(reversed_mapping_schoold)

# transform binary variables with 1 and 2 into 1 and 0
columns_to_transform = ["recog_sup","recog_effort", "recog_pay", "recog_personal" ,"gender", "overtime"]

# Iterate over the columns and replace the values 2 with 0
for col in columns_to_transform:
    recoded[col] = recoded[col].replace({2: 0})

# save df somewhere so its not muted when repeatedly executing this cell: Can later transform that into functions
dfnan = recoded
# remove rows with '3' in overtime
dfnan.drop(dfnan[dfnan['overtime'] == 3].index, inplace=True)

dfnan.head(3)

,,syear_x,take_revenge,similar_problems,insult_back,syear_y,year_birth,gender,overtime,wage_lastmonth,recog_sup,...,school_degree,sector,tenure,years_educ,sector_avg_wage,relative_wage,avg_rec,age,potential_experience,age_squared
pid,hid,,,,,,,,,,,,,,,,,,,,,
201,27,2005,1.0,5.0,1.0,2006,1926,0,NaN,NaN,NaN,...,[1] Hauptschulabschluss,NaN,NaN,10.0,94.115693,NaN,2.333333,80,3844,64.00
901,94,2005,5.0,3.0,3.0,2006,1951,0,1.0,1252.0,0.0,...,[1] Hauptschulabschluss,[52] Einzelhandel (ohne Handel mit Kraftfahrze...,5.0,10.0,943.961012,1.326326,3.666667,55,1369,30.25
1202,124,2005,1.0,1.0,1.0,2006,1913,0,NaN,NaN,NaN,...,[1] Hauptschulabschluss,NaN,NaN,10.0,94.115693,NaN,1.000000,93,5625,86.49


In [64]:
dfnan.columns

Index(['syear_x', 'take_revenge', 'similar_problems', 'insult_back', 'syear_y',
       'year_birth', 'gender', 'overtime', 'wage_lastmonth', 'recog_sup',
       'recog_effort', 'recog_personal', 'recog_pay', 'syear', 'working_hours',
       'firmsize', 'school_degree', 'sector', 'tenure', 'years_educ',
       'sector_avg_wage', 'relative_wage', 'avg_rec', 'age',
       'potential_experience', 'age_squared'],
      dtype='object')

In [65]:
df_regression = dfnan.drop(columns=['syear_x', 'similar_problems', 'take_revenge', 'insult_back','syear_y', 'recog_sup',
       'recog_personal', 'recog_pay', 'syear_y', 'year_birth', 'sector_avg_wage', 'age', 'sector', 'wage_lastmonth', 'school_degree', "syear"])

In [66]:
df_regression

,,gender,overtime,recog_effort,working_hours,firmsize,tenure,years_educ,relative_wage,avg_rec,potential_experience,age_squared
pid,hid,,,,,,,,,,,
201,27,0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,2.333333,3844,64.00
901,94,0,1.0,1.0,42.0,6.0,5.0,10.0,1.326326,3.666667,1369,30.25
1202,124,0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,1.000000,5625,86.49
1601,167,1,NaN,NaN,NaN,NaN,NaN,10.0,NaN,4.333333,2304,43.56
1602,167,0,0.0,1.0,NaN,7.0,1.0,NaN,0.164775,1.333333,729,20.25
...,...,...,...,...,...,...,...,...,...,...,...,...
8261002,826103,0,1.0,1.0,40.0,NaN,1.0,10.0,5.620742,4.333333,9,4.41
8261302,826138,0,NaN,NaN,NaN,NaN,NaN,13.0,NaN,1.000000,9,4.41
8261502,826154,0,NaN,NaN,NaN,NaN,NaN,13.0,NaN,3.333333,9,4.41


In [67]:
df_regression["interaction"] = df_regression["recog_effort"] * df_regression["avg_rec"]

In [68]:
df_regression = df_regression.dropna()

In [69]:
df_regression.columns

Index(['gender', 'overtime', 'recog_effort', 'working_hours', 'firmsize',
       'tenure', 'years_educ', 'relative_wage', 'avg_rec',
       'potential_experience', 'age_squared', 'interaction'],
      dtype='object')

In [70]:
from statsmodels.discrete.discrete_model import Probit

Y = df_regression["overtime"]
X = df_regression.drop(columns=["overtime"])
X = sm.add_constant(X)
model = Probit(Y, X.astype(float))
probit_model = model.fit()

probit_model.summary()

Optimization terminated successfully.
         Current function value: 0.493184
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:               overtime   No. Observations:                 7016
Model:                         Probit   Df Residuals:                     7004
Method:                           MLE   Df Model:                           11
Date:                Wed, 04 Jan 2023   Pseudo R-squ.:                 0.09359
Time:                        17:23:36   Log-Likelihood:                -3460.2
converged:                       True   LL-Null:                       -3817.5
Covariance Type:            nonrobust   LLR p-value:                4.076e-146
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -1.9959      0.182    -10.961      0.000      -2.353      -1.639
gender                   0.0327      0.039      0.831      0.406      -0.044       0.110
recog_effort            -0.4430      0.090     -4.910      0.000      -0.620      -0.266
working_hours            0.0281      0.002     13.318      0.000       0.024       0.032
firmsize                 0.0195      0.007      2.904      0.004       0.006       0.033
tenure                  -0.0049      0.002     -2.294      0.022      -0.009      -0.001
years_educ               0.0897      0.007     12.840      0.000       0.076       0.103
relative_wage            0.0404      0.014      2.871      0.004       0.013       0.068
avg_rec                 -0.0430      0.021     -2.041      0.041      -0.084      -0.002
potential_experience    -0.0024      0.000     -8.317      0.000      -0.003      -0.002
age_squared              0.1356      0.017      7.985      0.000       0.102       0.169
interaction              0.0558      0.025      2.190      0.029       0.006       0.106
========================================================================================
"""

In [75]:
latex_output = probit_model.summary().as_latex()
latex_output

'\\begin{center}\n\\begin{tabular}{lclc}\n\\toprule\n\\textbf{Dep. Variable:}        &     overtime     & \\textbf{  No. Observations:  } &     7016    \\\\\n\\textbf{Model:}                &      Probit      & \\textbf{  Df Residuals:      } &     7004    \\\\\n\\textbf{Method:}               &       MLE        & \\textbf{  Df Model:          } &       11    \\\\\n\\textbf{Date:}                 & Wed, 04 Jan 2023 & \\textbf{  Pseudo R-squ.:     } &  0.09359    \\\\\n\\textbf{Time:}                 &     17:30:09     & \\textbf{  Log-Likelihood:    } &   -3460.2   \\\\\n\\textbf{converged:}            &       True       & \\textbf{  LL-Null:           } &   -3817.5   \\\\\n\\textbf{Covariance Type:}      &    nonrobust     & \\textbf{  LLR p-value:       } & 4.076e-146  \\\\\n\\bottomrule\n\\end{tabular}\n\\begin{tabular}{lcccccc}\n                               & \\textbf{coef} & \\textbf{std err} & \\textbf{z} & \\textbf{P$> |$z$|$} & \\textbf{[0.025} & \\textbf{0.975]}  \\\\\n\\mid